<a href="https://colab.research.google.com/github/Rekulapelliushasree19/YoutubeTranscriptSummarizer/blob/main/Youtube_Transcript_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers torch sentencepiece

In [ ]:
%pip install youtube_transcript_api

In [ ]:
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi as yta
from urllib.parse import urlparse, parse_qs

In [ ]:
summarization = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
link = input("Please Enter the link to the YouTube Video: ")
url_data = urlparse(link)
print("url DATA=",url_data)
query = parse_qs(url_data.query)
print("query",query)
vid_id = query["v"][0]

Please Enter the link to the YouTube Video: https://www.youtube.com/watch?v=Nf-P-qNej3c
url DATA= ParseResult(scheme='https', netloc='www.youtube.com', path='/watch', params='', query='v=Nf-P-qNej3c', fragment='')
query {'v': ['Nf-P-qNej3c']}


In [ ]:
import textwrap

data = yta.get_transcript(vid_id)
print("DATA\n",data)
for item in data:
    wrapped_text = textwrap.fill(item['text'], width=100)


DATA
 [{'text': "- [Narrator] Meet Walmart's Alphabot.", 'start': 1.32, 'duration': 1.85}, {'text': 'This robot spends all day\npicking items for online orders', 'start': 4.38, 'duration': 3.42}, {'text': "at the company's market\nfulfillment centers,", 'start': 7.8, 'duration': 2.49}, {'text': "and it's key to speeding\nup Walmart's delivery.", 'start': 10.29, 'duration': 2.88}, {'text': '95% of its orders can be\npicked in under 12 minutes.', 'start': 13.17, 'duration': 4.77}, {'text': 'In the shipping war\nagainst Amazon and Target,', 'start': 17.94, 'duration': 2.55}, {'text': 'Walmart is leaning into the cutting edge,', 'start': 20.49, 'duration': 2.163}, {'text': 'investing in drone delivery', 'start': 23.73, 'duration': 1.68}, {'text': 'and automated fulfillment centers.', 'start': 25.41, 'duration': 2.22}, {'text': '- By 2025, 65% of our stores\nwill be serviced by automation.', 'start': 27.63, 'duration': 5.0}, {'text': "- [Narrator] Here's how the\ncountry's biggest retailer"

In [ ]:
transcript = ' '.join([value['text'] for value in data])
print("\nTRANSCRIPT\n", textwrap.fill(transcript, width=100))


TRANSCRIPT
 - [Narrator] Meet Walmart's Alphabot. This robot spends all day picking items for online orders at
the company's market fulfillment centers, and it's key to speeding up Walmart's delivery. 95% of its
orders can be picked in under 12 minutes. In the shipping war against Amazon and Target, Walmart is
leaning into the cutting edge, investing in drone delivery and automated fulfillment centers. - By
2025, 65% of our stores will be serviced by automation. - [Narrator] Here's how the country's
biggest retailer is going high tech to deliver on more than just speed. This market fulfillment
center or MFC is attached to a Walmart Supercenter more than six times its size. Of all the
company's e-commerce fulfillment methods, MFCs have the smallest footprint. - Walmart is the largest
grocer in the country. They have added many more different ways to get delivery, often using
groceries as sort of a linchpin, so that was a big evolution when they started to think of their
online business

In [ ]:
print("Length of Actual Transcript : ",len(transcript))

Length of Actual Transcript :  8932


In [ ]:
def split_text_for_tokens(text, max_tokens=512, tokenizer=None):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk) for chunk in chunks]
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
transcript_chunks = split_text_for_tokens(transcript, max_tokens=512, tokenizer=tokenizer)

summary_text = ''
for chunk in transcript_chunks:
    summary = summarization(chunk, max_length=150, min_length=30, do_sample=False)
    summary_text += summary[0]['summary_text'] + ' '

print("Full Summary:", summary_text)

Full Summary: Walmart is investing in drone delivery and automated fulfillment centers. 95 % of its orders can be picked in under 12 minutes. By 2025, 65 % of our stores will be serviced by automation. Target is building a network of facilities that would relieve local stores of sorting and delivery duties. walmart now has seven of these facilities with plans to open over 100 more in the next few years.  walmart is also investing in five large facilities that it calls next generation fulfillment centers. at 1. 5 million square feet, these centers are fully automated. walmart + members pay $ 98 annually for free shipping with fees for delivery within hours. Walmart says it's not shrinking its workforce overall. The company is investing in automation so that it can deliver undamaged packages. walmart still has a long way to go in terms of catching up with amazon on e - commerce. 


In [ ]:
wrapped_summary = textwrap.fill(summary_text, width=100)
print("Summary:", wrapped_summary)

Summary: Walmart is investing in drone delivery and automated fulfillment centers. 95 % of its orders can be
picked in under 12 minutes. By 2025, 65 % of our stores will be serviced by automation. Target is
building a network of facilities that would relieve local stores of sorting and delivery duties.
walmart now has seven of these facilities with plans to open over 100 more in the next few years.
walmart is also investing in five large facilities that it calls next generation fulfillment
centers. at 1. 5 million square feet, these centers are fully automated. walmart + members pay $ 98
annually for free shipping with fees for delivery within hours. Walmart says it's not shrinking its
workforce overall. The company is investing in automation so that it can deliver undamaged packages.
walmart still has a long way to go in terms of catching up with amazon on e - commerce.


In [ ]:
print("Length of the Summarized Transcript : ",len(wrapped_summary))

Length of the Summarized Transcript :  874


In [ ]:
with open("summary.txt", "w") as text_file:
    text_file.write(wrapped_summary)
